# Data scales from simulated data.

In this notebook i'll be checking the scales on the simulated data to see which weights i could use for the non-dimensional equations on the physics informed loss so that they balance well.

In [1]:
import fluidfoam as ff
import numpy as np
import matplotlib.pyplot as plt

In [23]:
rho = 1
mu = 0.001
alpha = 0.005
D = 0.1
L = D
beta = 1.0
g = 9.81

T0 = 300
T_bottom = 301

S0 = (T0 - T0)/(T_bottom - T0)
S_bottom = (T_bottom - T0)/(T_bottom - T0)


assert T_bottom > T0, "For a Benard cell the bottom temperature should be higher than the rest"

Ra = g*beta*(T_bottom-T0)*D**3*rho/(alpha*mu)
Pr = mu/(rho*alpha)
Gr = g*beta*(T_bottom - T0)*L**3*rho**2/mu**2
print(f"{Ra = }, {Pr = }, {Gr = }")

Ra = 1962.0000000000005, Pr = 0.2, Gr = 9810.000000000004


In [55]:
case = "vizualizing_simulation_data/bernard_cell_of_v2/"
x, y, z = ff.readmesh(case)
u, v, w = ff.readfield(case, "1000", "U")
T = ff.readscalar(case, "1000", "T")
p = ff.readscalar(case, "1000", "p")

Reading file vizualizing_simulation_data/bernard_cell_of_v2//constant/polyMesh/owner
Reading file vizualizing_simulation_data/bernard_cell_of_v2//constant/polyMesh/faces
Reading file vizualizing_simulation_data/bernard_cell_of_v2//constant/polyMesh/points
Reading file vizualizing_simulation_data/bernard_cell_of_v2//constant/polyMesh/neighbour
Reading file vizualizing_simulation_data/bernard_cell_of_v2/1000/U
Reading file vizualizing_simulation_data/bernard_cell_of_v2/1000/T
Reading file vizualizing_simulation_data/bernard_cell_of_v2/1000/p


In [48]:
U.max()

9.056004625065906e-05

# Scalling unities to the non-dimensional range

In [72]:
def transf_U(U):
    return U/np.sqrt(g*beta*(T_bottom - T0)*L)

def transf_T(T):
    return (T-T0)/(T_bottom - T0)

def transf_P(p):
    return p/(rho*g*beta*(T_bottom - T0)*L)

U = transf_U(u)
V = transf_U(v)
TT = transf_T(T)
P = transf_P(p)

fields = [U, V, P, TT]

In [57]:
TT.max()

0.9820000000000277

In [80]:
quantities = np.array([abs(f.mean()) for f in fields])

total = sum(quantities)

fractions = quantities/total

weights = 1/fractions

fractions, weights

(array([1.17744871e-11, 7.01485626e-09, 8.23708898e-01, 1.76291095e-01]),
 array([8.49293895e+10, 1.42554596e+08, 1.21402112e+00, 5.67243628e+00]))

In [74]:
[(f/f.max()).mean() for w,f in zip(weights, fields)], [(w*f).mean() for w,f in zip(weights, fields)]

([6.580206442647896e-08,
  3.14487150193184e-05,
  -2114.3325591661123,
  0.09085588085539402],
 [6.464398861412147e-08,
  3.089523700132567e-05,
  -2077.121608146122,
  0.08925687330204982])

In [79]:
P.max(), P.min()

(0.00019716717635066255, -0.9081264016309887)

array([1.08480578e+03, 8.70239111e+02, 1.00040796e-01, 4.98257689e+02])

In [39]:
(U*weights[0]).mean()

6.464398861412147e-08

In [41]:
U.mean()

5.959037997879842e-12